In [3]:
from h2oai_client import Client
import getpass

In [4]:
import h2oai_client
h2oai_client.__version__

'1.8.2'

In [12]:
address = 'http://ip_where_driverless_is_running:12345'
username = 'username'
password = getpass.getpass()
h2oai = Client(address = address, username = username, password = password)

········


### Custom function to start experiment asynchronously

In [13]:
from h2oai_client import Model, ModelParameters, DatasetReference, ModelReference

def my_start_experiment(self, dataset_key: str, target_col: str, is_classification: bool,
                          accuracy: int, time: int, interpretability: int,
                          scorer=None,
                          score_f_name: str = None, **kwargs) -> Model:
 
    if scorer is not None:
        score_f_name = scorer
    params = ModelParameters(
        dataset=DatasetReference(dataset_key),
        resumed_model=ModelReference(
            kwargs.get('resumed_model_key', '')),
        target_col=target_col,
        weight_col=kwargs.get('weight_col', None),
        fold_col=kwargs.get('fold_col', None),
        orig_time_col=kwargs.get('orig_time_col',
            kwargs.get('time_col', None)),
        time_col=kwargs.get('time_col', None),
        is_classification=is_classification,
        cols_to_drop=kwargs.get('cols_to_drop', []),
        validset=DatasetReference(
            kwargs.get('validset_key', '')),
        testset=DatasetReference(
            kwargs.get('testset_key', '')),
        enable_gpus=kwargs.get('enable_gpus', True),
        seed=kwargs.get('seed', False),
        accuracy=accuracy,
        time=time,
        interpretability=interpretability,
        score_f_name=score_f_name,
        time_groups_columns=kwargs.get('time_groups_columns', None),
        unavailable_columns_at_prediction_time=kwargs.get(
            'unavailable_columns_at_prediction_time', []),
        time_period_in_seconds=kwargs.get('time_period_in_seconds', None),
        num_prediction_periods=kwargs.get('num_prediction_periods', None),
        num_gap_periods=kwargs.get('num_gap_periods', None),
        is_timeseries=kwargs.get('is_timeseries', False),
        cols_imputation=kwargs.get('cols_imputation', []),
        config_overrides=kwargs.get('config_overrides', None),
        #custom_features=[],
    )
    # If custom recipes acceptance jobs are running, wait for them to finish
    if not kwargs.get('force_skip_acceptance_tests', False):
        self._wait_for_custom_recipes_acceptance_tests()
    key = self.start_experiment(params, kwargs.get('experiment_name', ''))
    #job = self._wait_for_model(key)
    return key#job.entity

### Create Functions that finds datasets

In [14]:
def find_dataset_keys(dataset_dump, dataset_name):
    keys = []

    for dataset in dataset_dump['datasets']:
        if dataset['name'] == dataset_name:
            keys.append(dataset['key'])
            
    return keys

### Get Dataset Key

In [16]:
#Get dataset_dump
dataset_dump = h2oai.list_datasets(offset=0, limit=10000, include_inactive=True).dump()  

#Upload training data if not present
keys = find_dataset_keys(dataset_dump, 'cc_train')
if len(keys) > 0:
    train_key = keys[0]     
    print('Training data already present. Using key:', train_key)

## Run Experiment

In [24]:
#Start experiment
experiment_key = my_start_experiment(h2oai, 
                                         dataset_key=train_key,
                                         target_col="Amount",
                                         is_classification=False,
                                         accuracy=5, 
                                         time=5, 
                                         interpretability=5, 
                                         scorer="RMSE")
print('Started experiment for key:', experiment_key)    

Started experiment for key: 630c31a2-d6c7-11ea-8496-ac1f6b68b7be
